Universidad del Valle de Guatemala <br>
04/03/2022 <br>
Minería de Datos # 20 

# Hoja de Trabajo 3
## Árboles de Decisión

**Integrantes:**<br>
Walter Saldaña 19897 <br>
José Gutierrez 19111 <br>
Javier Cotto 19324 <br>